<a href="https://colab.research.google.com/github/anmol-sinha-coder/DEmoClassi/blob/master/Age_Gender_Race_Emotion_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/G_Drive')

In [ ]:
! git clone https://github.com/anmol-sinha-coder/DEmoClassi.git
! cp -ra DEmoClassi/{vision_utils,emotion_detection,multitask_rag,'setup.py'} ./
! pip install tensorboardX pytorch-ignite pillow
! unzip /content/G_Drive/MyDrive/ADNN/facial-expression-recognition-challenge.zip -d .
! tar -xzvf /content/G_Drive/MyDrive/ADNN/UTKFace/UTKFace.tar.gz -C .
! tar -xzvf fer2013.tar.gz
! cp /content/G_Drive/MyDrive/ADNN/cv2_gpu/cv2.cpython-36m-x86_64-linux-gnu.so .

In [ ]:
import torch
import torchvision.transforms as transforms

from vision_utils.custom_torch_utils import load_model
from vision_utils.custom_architectures import SepConvModelMT, SepConvModel, initialize_model

from emotion_detection.evaluate import evaluate_model as eval_fer
from emotion_detection.fer_data_utils import get_fer_dataloader
from emotion_detection.train import run_fer

from multitask_rag.train import run_utk
from multitask_rag.utk_data_utils import get_utk_dataloader
from multitask_rag.evaluate import evaluate_model as eval_utk
from multitask_rag.utk_data_utils import display_examples_utk

import pandas as pd
import glob
import os
import random
import cv2
cv2.__version__